In [1]:
"""
This is the sentiment analysis with

Stopword, punctuation removal + mark negation

"""

In [2]:
import os
import nltk
import random
from nltk.tokenize import RegexpTokenizer
from nltk.text import Text
from nltk.classify import ClassifierI
import string, re
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from statistics import mode
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import mark_negation



documents = []
all_words = []
tokenizer = RegexpTokenizer(r'\w+')
sentim_analyzer = SentimentAnalyzer() # use nltk library
documents = [nltk.sentiment.util.mark_negation(doc) for doc in documents]


positive_files_path = "./review_polarity/txt_sentoken/pos/"
negative_files_path = "./review_polarity/txt_sentoken/neg/"

for root, dirs, files in os.walk(positive_files_path, topdown=False):
    for file in files:
        filepath = os.path.join(root, file)
        text = open(filepath, 'r').read()
        words = tokenizer.tokenize(text)
        mark_negation(words)
        
        documents.append((words, 'pos'))
        all_words.extend(words)

for root, dirs, files in os.walk(negative_files_path, topdown=False):
    for file in files:
        filepath = os.path.join(root, file)
        text = open(filepath, 'r').read()
        words = tokenizer.tokenize(text)
        mark_negation(words)
        
        documents.append((words, 'neg'))
        all_words.extend(words)
                

In [3]:
#function that finds features from a document
#features are top 20% most frequent words from all documents

all_words = nltk.FreqDist(all_words)
top_10 = int(len(all_words)*.20)
word_features, frequency = map(list, zip(*all_words.most_common(top_10)))

def find_features(document):
    words = set(document)
    features = {}
    for word in word_features:
        features[word] = (word in words)
        
    return features

In [4]:
featuresets = [(find_features(review), category) for (review, category) in documents]
random.shuffle(featuresets)

training_set = featuresets[500:] # 1500
testing_set = featuresets[:500] # 500

In [5]:
names = ['MultinomialNB','BernoulliNB', 
         'Decision_Tree',
         'Logistic Regression', 'Stochastic Gradient Descent',
         'SVC', 'LinearSVC', 'NuSVC']

trained = []

classifiers = [MultinomialNB(), BernoulliNB(), 
               DecisionTreeClassifier(random_state=0), 
               LogisticRegression(), SGDClassifier(),
               SVC(), LinearSVC(), NuSVC()]

for name, clf in zip(names, classifiers):
    print('Now classifying', name)
    
    classifier = SklearnClassifier(clf)
    classifier.train(training_set)
    trained.append(classifier)
    print("{} classifier accuracy:".format(name), nltk.classify.accuracy(classifier, testing_set))
    print("\n")

Now classifying MultinomialNB
MultinomialNB classifier accuracy: 0.806


Now classifying BernoulliNB
BernoulliNB classifier accuracy: 0.79


Now classifying Decision_Tree
Decision_Tree classifier accuracy: 0.61


Now classifying Logistic Regression
Logistic Regression classifier accuracy: 0.848


Now classifying Stochastic Gradient Descent
Stochastic Gradient Descent classifier accuracy: 0.842


Now classifying SVC
SVC classifier accuracy: 0.84


Now classifying LinearSVC


C:\Users\YunMo\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC classifier accuracy: 0.826


Now classifying NuSVC
NuSVC classifier accuracy: 0.836




In [6]:
def analyze_text(trained, input_text):
    text = tokenizer.tokenize(input_text)
    features = find_features(text)
    """
    results = []
    for classifier in trained:
        results.append(classifier.classify(features))
        print(results)
    return mode(results)
    """
    return trained[3].classify(features)

In [7]:
#input the text you want sentiment analyzed in here:
#after inputting, please press shift + enter to run this cell.

sentiment_text = "This film was awesome, it took me back to when I was a child playing with my millennium falcon at home. It's fan service at its purest and repaired all the damage from the last film. Don't watch this film with a predetermined view of how you think this film should be, and watch it through a child's eyes as you did the first time you watched this franchise and just enjoy."

print("your input's sentiment is: ", analyze_text(trained, sentiment_text))

your input's sentiment is:  pos
